# Procesamiento de lenguaje natural

### Alumno
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Utilizar modelos 
    - Skip-Gram - Entrenamiento
    - CBOW - Entrenamiento
- Graficarlos.
- Obtener conclusiones.

In [66]:
%reset -f
# !pip install gensim

### contenido

El presente notebook se centra en la práctica de embeddings de palabras utilizando la librería Gensim y el algoritmo Word2Vec sobre un corpus literario de ciencia ficción la Trilogía de la Fundación de Isaac Asimov.




In [67]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import struct; print("Bits", 8 * struct.calcsize("P"))
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import gensim; print("gemsim", gensim.__version__)
import tensorflow ; print ("tensorflow", tensorflow.__version__)

""" 
Recomendado para plataforma Windows-11-10.0.26100-SP0
Python 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
Bits 64
NumPy 1.26.4
SciPy 1.13.1
gemsim 4.3.3
tensorflow 2.19.0
"""

Windows-11-10.0.26100-SP0
Python 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
Bits 64
NumPy 1.26.4
SciPy 1.13.1
gemsim 4.3.3
tensorflow 2.18.0


' \nRecomendado para plataforma Windows-11-10.0.26100-SP0\nPython 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]\nBits 64\nNumPy 1.26.4\nSciPy 1.13.1\ngemsim 4.3.3\ntensorflow 2.19.0\n'

<div style="text-align: right;">
	<h1 style="display: inline-block;margin: 0;padding: 8px 16px;color: white;background: linear-gradient(to right,rgb(17, 75, 141), #4CAF50);border-radius: 12px;font-size: 1.8rem;box-shadow: 0 4px 10px rgba(0, 0, 0, 0.3);">nota</h1>
</div>



## Custom embedddings con Gensim

## Conceptos Introductorios en PLN: Embeddings, Word2Vec y Gensim

Para que las computadoras puedan procesar y "entender" el lenguaje humano, las palabras deben transformarse en un formato numérico. Aquí es donde entran en juego los **embeddings**.

### 1. Embeddings de Palabras (Introducción General)

- **¿Qué son?** Los embeddings de palabras son **representaciones numéricas densas** de palabras, donde cada palabra se convierte en un **vector** (una lista de números decimales, como `[0.123, -0.456, 0.789, ...]`). Estos vectores suelen tener una dimensionalidad mucho menor (ej., 50, 100, 300) que el vocabulario completo, lo que los hace eficientes.
    
- **La Idea Clave:** La magia de los embeddings radica en que **palabras con significados similares o que aparecen en contextos parecidos se ubican cerca unas de otras** en este espacio vectorial multidimensional. La "distancia" (ej., similitud coseno) y la "dirección" entre estos vectores capturan relaciones semánticas (ej., "rey" está cerca de "reina") y sintácticas.
    
- **¿Para qué se utilizan?** Permiten que los algoritmos de Machine Learning (que solo entienden números) trabajen con texto de una manera más rica y significativa que los métodos tradicionales como la "bolsa de palabras". Son fundamentales para tareas modernas de PLN como análisis de sentimiento, traducción, clasificación de texto, y más.
    

---

### 2. Word2Vec

- **¿Qué es?** **Word2Vec** es un **algoritmo popular** (y una familia de modelos) desarrollado por Google que se utiliza para **aprender embeddings de palabras**. No es un embedding en sí mismo, sino el método para crearlos. Se entrena en grandes colecciones de texto (corpus) y aprende a asociar palabras con sus contextos.
    
- **¿Cómo funciona (idea simplificada)?** Word2Vec se basa en la "hipótesis distribucional": las palabras que aparecen en contextos similares tienden a tener significados similares. Internamente, utiliza redes neuronales poco profundas para dos tareas principales:
    
    - **Skip-gram:** Dado una palabra, predice las palabras de su contexto (las palabras que la rodean).
        
    - **CBOW (Continuous Bag-of-Words):** Dado el contexto de una palabra (las palabras que la rodean), predice la palabra central.
        
- **Resultado:** Después de entrenar Word2Vec en un corpus, el resultado final es un "modelo" que contiene los embeddings (vectores) para cada palabra en el vocabulario del corpus.
    

---

### 3. Gensim

- **¿Qué es?** **Gensim** es una **librería de Python** especializada en el modelado de temas (Topic Modeling), la similitud de documentos y, muy importante para nuestro contexto, la **creación y manipulación de embeddings de palabras** (como Word2Vec).
    
- **Características Principales:**
    
    - **Eficiencia:** Está diseñada para manejar grandes volúmenes de texto de manera eficiente, lo cual es crucial en PLN.
        
    - **Implementación de Algoritmos:** Proporciona implementaciones de algoritmos clave como LDA, LSI, y por supuesto, Word2Vec y FastText.
        
    - **Simplicidad:** Ofrece una interfaz relativamente sencilla para trabajar con estos complejos modelos.
        
- **¿Para qué se utiliza?**
    
    - Entrenar tus propios modelos Word2Vec, FastText o Doc2Vec a partir de tu propio corpus de texto.
        
    - Cargar y utilizar modelos de embeddings pre-entrenados (como los de Google News que contienen embeddings de Word2Vec).
        
    - Realizar tareas de modelado de temas como Latent Dirichlet Allocation (LDA).
        
    - Calcular la similitud entre palabras o documentos utilizando los embeddings aprendidos.
        

---

### 4. Embeddings con Gensim

- **¿Qué significa?** "Embeddings con Gensim" se refiere al **proceso de generar y trabajar con embeddings de palabras o documentos utilizando la librería Gensim**. Gensim facilita todo el ciclo de vida de los embeddings:
    
    1. **Preparación del Corpus:** Gensim ayuda a preprocesar tu texto (tokenización, construcción de vocabulario, etc.) en un formato que sus modelos puedan entender.
        
    2. **Entrenamiento del Modelo:** Utilizas clases como `gensim.models.Word2Vec` o `gensim.models.FastText` para entrenar un modelo de embeddings en tu corpus. Durante este entrenamiento, el algoritmo aprende los vectores de las palabras.
        
    3. **Acceso a los Embeddings:** Una vez entrenado, el objeto modelo de Gensim te permite acceder a los vectores (embeddings) de las palabras (`model.wv['palabra_ejemplo']`).
        
    4. **Operaciones con Embeddings:** Gensim te proporciona métodos convenientes para realizar operaciones comunes con los embeddings, como encontrar las palabras más similares (`model.wv.most_similar('palabra')`), realizar analogías, etc.

---




In [68]:
%reset -f

In [69]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# PNL
import multiprocessing
# que es Word2Vec
from gensim.models import Word2Vec
#Complemento 
import os

# Custom embedddings con Gensim -> Word2Vec

## Carga del DataSet 

Dentro de la carpeta  `dataset_novelas-txt` tenemos nuestro corpus en idioma español y formato .txt .

Para nuestro ejercicio escogemos uno de los archivos .txt como dataset. Los archivos están estructurados con un párrafo por línea y sin líneas vacías. 

En código a continuación  utilizamos `sep='/n'` para dividir el texto por saltos de línea permitiendo tratar cada párrafo como un documento independiente, de esta forma tendremos suficiente contexto semántico y el modelo Word2Vec puede aprender las relaciones entre palabras y generar word embeddings

In [70]:
# cargamos el Dataset 
os.listdir("./dataset_novelas-txt/")
df = pd.read_csv('./dataset_novelas-txt/Trilogía-Fundación-_Ed.-ilustrada_-Isaac-Asimov.txt', sep='/n', header=None)

C:\Users\SAN\AppData\Local\Temp\ipykernel_24628\2829754718.py:3: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



## EDA del dataset 
Realizamos un pequeño análisis exploratorio para conocer parte del contenido del dataset (archivo .txt) donde vemos :
- Esta conformado por un dataset de texto sin etiquetas tomamos en cuenta que el Word2Vec no necesita etiquetas, para un aprendizaje no supervisado
- Todo el corpus esta dividido en 8455 documentos 


In [71]:
# Cantidad de documentos 
print(f"*. Se encuentra conformado por {df.shape[0]} documentos")

*. Se encuentra conformado por 8455 documentos


In [72]:
# mostramos los primeros 5 documentos 
print (f"*. los primero 5 documentos son : \n {df.head()}")


*. los primero 5 documentos son : 
                                                    0
0  Isaac Asimov empezó a escribir la saga de la F...
1  ¿Por qué surgen y caen los imperios? La Trilog...
2  Esta edición única de la Trilogía Fundación, i...
3                                       Isaac Asimov
4                                 Trilogía Fundación


In [73]:
# estructura del  dataset:
print("Estructura del dataset:")
print(f"- Forma: {df.shape}")
print(f"- Columnas: {df.columns.tolist()}")
print(f"- Tipo de datos: {df.dtypes}")

# Análisis de longitud de documentos
df['longitud'] = df[0].str.len()
print(f"- Longitud promedio por documento: {df['longitud'].mean():.1f} caracteres")
print(f"- Documento más corto: {df['longitud'].min()} caracteres")
print(f"- Documento más largo: {df['longitud'].max()} caracteres")

# Ejemplos de diferentes tipos de contenido
print("\nTipos de contenido encontrados:")
print("Documentos cortos (probablemente metadatos):")
print(df[df['longitud'] < 50][0].head(10))

Estructura del dataset:
- Forma: (8455, 1)
- Columnas: [0]
- Tipo de datos: 0    object
dtype: object
- Longitud promedio por documento: 157.1 caracteres
- Documento más corto: 1 caracteres
- Documento más largo: 1255 caracteres

Tipos de contenido encontrados:
Documentos cortos (probablemente metadatos):
3                                      Isaac Asimov
4                                Trilogía Fundación
5                                 Edición Ilustrada
6                                         ePub r1.0
7                                Watcher 01-10-2022
8           Título original: The Foundation Trilogy
9                                Isaac Asimov, 1963
10    Traducción: Manuel de los Reyes García Campos
12                            Colección NOVA nº 337
13                          Editor digital: Watcher
Name: 0, dtype: object


In [74]:
# muestra 5 documentos que contengan menos de 1 carácter 
print(df[df['longitud'] < 2][0].head(5))

85     1
111    2
150    3
218    4
277    5
Name: 0, dtype: object


In [75]:
# consulta de documentos por posición 
print(f"Documentos que se encuentran entre las posiciones 200 al 205: \n {df[149:152]}")

Documentos que se encuentran entre las posiciones 200 al 205: 
                                                      0  longitud
149  En todo este tiempo, desde el momento del dese...       105
150                                                  3         1
151  TRANTOR: […] Fue a comienzos del decimotercer ...       439


## Preprocesamiento

Realizamos limpieza de los datos donde utilizamos el `text_to_word_sequence` para convertir la cadena de texto en una lista de palabras en minúscula y sin puntuación ejemplo 
-  "Fundación" y "fundación" ------>  "fundacion"
- "exclamó" -----> exclamo

In [76]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [77]:
# "Dame todos los elementos de la lista sentence_tokens desde el principio (índice 0) hasta el índice 1, sin incluir el elemento en el índice 1."
sentence_tokens[:1]
#sentence_tokens[2:3]

[['isaac',
  'asimov',
  'empezó',
  'a',
  'escribir',
  'la',
  'saga',
  'de',
  'la',
  'fundación',
  'a',
  'los',
  'veintiún',
  'años',
  'sin',
  'saber',
  'que',
  'un',
  'día',
  'esta',
  'obra',
  'se',
  'convertiría',
  'en',
  'la',
  'piedra',
  'angular',
  'de',
  'la',
  'ciencia',
  'ficción',
  'del',
  'siglo',
  'xx',
  'ni',
  'que',
  'con',
  'ella',
  'cautivaría',
  'a',
  'lectores',
  'de',
  'todas',
  'las',
  'edades',
  'durante',
  'más',
  'de',
  'siete',
  'décadas']]

In [78]:
total_palabras = 0
for sentence in sentence_tokens:
    total_palabras += len(sentence)

print(f"El número total de palabras en 'sentence_tokens' es: {total_palabras} (tomando en cuenta la duplicidad)")

## Otra forma para contar la lista de palabras sum()
#total_palabras_conciso = sum(len(sentence) for sentence in sentence_tokens)
#print(f"El número total de palabras  es: {total_palabras_conciso}")

El número total de palabras en 'sentence_tokens' es: 221985 (tomando en cuenta la duplicidad)


## Creación del modelo Word2Vec


En el siguiente bloque de código definimos un callback personalizado para los modelos de embeddings de Gensim (Word2Vec). Su propósito principal es monitorear y reportar la pérdida (loss) de entrenamiento en cada época durante el proceso de entrenamiento del modelo, ya que Gensim no lo hace por defecto de una manera fácilmente visible.


La pérdida (loss) es un valor numérico que cuantifica el error o la inexactitud de las predicciones de un modelo. Cuanto mayor sea su valor, peores serán las predicciones del modelo; inversamente, cuanto menor sea la pérdida, mejor será la predicción.


**Cómo funciona según la arquitectura**

|Arquitectura|Tarea de Predicción|Función de Pérdida Evalúa|
|---|---|---|
|**Skip-gram**|Predice palabras de contexto → objetivo|Efectividad al predecir vecinos de una palabra central|
|**CBOW**|Predice palabra central ← contexto|Efectividad al predecir palabra central desde el contexto|



In [79]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self, model_name=""):
        self.epoch = 0
        self.model_name = model_name  # Para identificar qué modelo está entrenando
        self.loss_previous_step = 0

    def on_epoch_end(self, model):
         #  Este método de Gensim te devuelve la pérdida acumulada desde la última vez que el estado de entrenamiento del modelo fue "reiniciado"
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print(f'{self.model_name} - Loss after epoch {self.epoch}: {loss}')
        else:
            print(f'{self.model_name} - Loss after epoch {self.epoch}: {loss - self.loss_previous_step}')
        self.epoch += 1
        self.loss_previous_step = loss

### Entrenar embeddings

Utilizaremos Word2Vec, un algoritmo para aprender embeddings de palabras (vectores numéricos densos), a partir de nuestro corpus de texto. Este proceso generará representaciones vectoriales para cada palabra. 


<b id="Word2Vec">Word2Vec tiene dos arquitecturas</b> principales para aprender las representaciones vectoriales de las palabras en este ejercicio utilizaremos:

1. **CBOW (Continuous Bag-of-Words):**
    
    - **Funcionamiento:** Toma las palabras que rodean a una palabra central (el **contexto**) como entrada y predice la **palabra central**.
        
    - **Ventajas:** Es más rápido de entrenar y funciona muy bien con palabras frecuentes.
        
    - **Parámetro en Gensim:** `sg = 0` (el valor por defecto).
        
2. **Skip-gram:**
    
    - **Funcionamiento:** Toma una **palabra central** como entrada y predice las **palabras de contexto** que la rodean. Es el opuesto de CBOW.
        
    - **Ventajas:** Es mejor para capturar relaciones semánticas complejas y funciona bien con palabras raras o que aparecen con poca frecuencia.
        
    - **Parámetro en Gensim:** `sg = 1`.



In [80]:
# modelo Word2Vec
print("="*60)
print("ENTRENANDO MODELO SKIP-GRAM (sg=1)")


# Modelo 1: Skip-gram (sg=1)
w2v_skipgram = Word2Vec(min_count=5,        #  Ignora palabras que aparecen menos de 5 veces (reduce ruido)
                        window=2,           # Contexto de 2 palabras a cada lado (antes y después como contexto )
                        vector_size=300,    # Cada palabra será un vector de 300 dimensiones
                        negative=20,        # Técnica de optimización (muestreo negativo)
                        workers=1,          # cuanto CPU utilizar 
                        sg=1)  # ARQUITECTURA/TIPO Skip-gram

# pasamos el vocabulario al modelo
w2v_skipgram.build_vocab(sentence_tokens)

print(f"Skip-gram - Cantidad de documentos en el corpus: {w2v_skipgram.corpus_count}")
print(f"Skip-gram - Cantidad de palabras en vocabulario: {len(w2v_skipgram.wv.index_to_key)}")

# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_skipgram.train(sentence_tokens,
                   total_examples=w2v_skipgram.corpus_count,
                   epochs=20,
                   compute_loss=True,
                   callbacks=[callback("SKIP-GRAM")])

print("ENTRENAMIENTO COMPLETADO")
print("="*60)

ENTRENANDO MODELO SKIP-GRAM (sg=1)
Skip-gram - Cantidad de documentos en el corpus: 8455
Skip-gram - Cantidad de palabras en vocabulario: 4644
SKIP-GRAM - Loss after epoch 0: 1782827.0
SKIP-GRAM - Loss after epoch 1: 1167566.5
SKIP-GRAM - Loss after epoch 2: 1110558.0
SKIP-GRAM - Loss after epoch 3: 1046721.0
SKIP-GRAM - Loss after epoch 4: 1027282.5
SKIP-GRAM - Loss after epoch 5: 1017644.5
SKIP-GRAM - Loss after epoch 6: 1002948.0
SKIP-GRAM - Loss after epoch 7: 967019.5
SKIP-GRAM - Loss after epoch 8: 949066.0
SKIP-GRAM - Loss after epoch 9: 940482.0
SKIP-GRAM - Loss after epoch 10: 931070.0
SKIP-GRAM - Loss after epoch 11: 922380.0
SKIP-GRAM - Loss after epoch 12: 912506.0
SKIP-GRAM - Loss after epoch 13: 906706.0
SKIP-GRAM - Loss after epoch 14: 901101.0
SKIP-GRAM - Loss after epoch 15: 895594.0
SKIP-GRAM - Loss after epoch 16: 872002.0
SKIP-GRAM - Loss after epoch 17: 860788.0
SKIP-GRAM - Loss after epoch 18: 855154.0
SKIP-GRAM - Loss after epoch 19: 855004.0
ENTRENAMIENTO COMPLE

In [81]:
print("="*60)
print("ENTRENANDO MODELO CBOW (sg=0)")

# Modelo 2: CBOW (sg=0)
w2v_cbow = Word2Vec(min_count=5,    
                    window=2,       
                    vector_size=300,       
                    negative=20,    
                    workers=1,   
                    sg=0)  # CBOW

w2v_cbow.build_vocab(sentence_tokens)

print(f"CBOW - DCantidad de documentos en el corpus: {w2v_cbow.corpus_count}")
print(f"CBOW - Cantidad de palabras en vocabulario: {len(w2v_cbow.wv.index_to_key)}")
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_cbow.train(sentence_tokens,
               total_examples=w2v_cbow.corpus_count,
               epochs=20,
               compute_loss=True,
               callbacks=[callback("CBOW")])

print("ENTRENAMIENTO COMPLETADO")
print("="*60)

ENTRENANDO MODELO CBOW (sg=0)
CBOW - DCantidad de documentos en el corpus: 8455
CBOW - Cantidad de palabras en vocabulario: 4644
CBOW - Loss after epoch 0: 766093.75
CBOW - Loss after epoch 1: 543693.5
CBOW - Loss after epoch 2: 494966.375
CBOW - Loss after epoch 3: 449346.875
CBOW - Loss after epoch 4: 397442.75
CBOW - Loss after epoch 5: 389348.0
CBOW - Loss after epoch 6: 380808.5
CBOW - Loss after epoch 7: 374795.5
CBOW - Loss after epoch 8: 368295.25
CBOW - Loss after epoch 9: 341653.5
CBOW - Loss after epoch 10: 335403.5
CBOW - Loss after epoch 11: 330135.5
CBOW - Loss after epoch 12: 326310.0
CBOW - Loss after epoch 13: 322524.5
CBOW - Loss after epoch 14: 319547.5
CBOW - Loss after epoch 15: 317019.5
CBOW - Loss after epoch 16: 314050.5
CBOW - Loss after epoch 17: 312077.5
CBOW - Loss after epoch 18: 310907.0
CBOW - Loss after epoch 19: 308851.5
ENTRENAMIENTO COMPLETADO


### Exploración de similitudes 


- Positive: Encuentra palabras  que aparecen en contexto similares a "imperio"
- Negative: Encuentra palabras  que aparecen en contexto diferentes a "imperio"

Después del entrenamiento, cada palabra tiene un vector de 300 números que captura su "significado" basado en sus contextos. Palabras con significados similares tendrán vectores similares(cercano) en el espacio multidimensional.

Word2Vec aprendió que "imperio", "espíritu", "galáctico" aparecen en contextos similares de acuerdo al dataset (Fundación), por lo que sus vectores estarán cerca en el espacio vectorial en Word2Vec



#### Consulta de similitudes positive

In [102]:
# modelo SKIP-GRAM -> Similitudes positivas, encontramos 5 palabras similares 
w2v_skipgram.wv.most_similar(positive=["imperio"], topn=5)
# Mientras el resultado este mas cercano a 1 será mas fuerte la relación del valor de Similitud coseno


[('espíritu', 0.6739826202392578),
 ('galáctico', 0.5707151889801025),
 ('universal', 0.5567495822906494),
 ('fundacionista', 0.5410391688346863),
 ('lugar', 0.5362286567687988)]

In [109]:
# modelo CBOW -> Similitudes positivas, encontramos 5 palabras similares 
w2v_cbow.wv.most_similar(positive=["imperio"], topn=5)


[('espíritu', 0.8241751790046692),
 ('fundacionista', 0.6728864908218384),
 ('reino', 0.6250720024108887),
 ('universo', 0.6210659146308899),
 ('modelo', 0.6205688714981079)]

Como podemos observar, ambos modelos **Skip-gram** y **CBOW** arrojan resultados con algunas palabras distintas o en distinto orden ("espíritu" , "fundacionista"). Ambos modelos aprenden de manera diferente, por lo que generan vectores distintos para las mismas palabras


**Proceso de Aprendizaje**
Skip-gram (sg=1):
- Entrada: Una palabra central ("imperio")
- Salida: Predice las palabras del contexto circundante
- Aprende: "Si veo 'imperio', ¿qué palabras espero encontrar alrededor?"
- Sensibilidad al contexto: Más sensible a palabras frecuentes en contextos específicos
- Skip-gram: Mejor para palabras específicas/raras

CBOW (sg=0):
- Entrada: Palabras del contexto circundante
- Salida: Predice la palabra central
- Aprende: "Si veo estas palabras alrededor, ¿cuál debería ser la palabra central?"
- Sensibilidad al contexto: Mejor para capturar el significado promedio del contexto
- CBOW: Mejor para conceptos generales/frecuentes




In [113]:
# similitud positiva, prueba con multiples palabras
palabras_test = ['psicohistoria', 'fundacionista', 'espíritu', 'planeta']

print("CBOW similitud -> positive:")
for palabra in palabras_test:
    try:
        # por defecto realizamos una similitud positiva 
        # w2v_cbow.wv.most_similar(positive=[palabra]
        print(f"{palabra}: {w2v_cbow.wv.most_similar(palabra, topn=5)}")
    except:
        print(f"{palabra}: no encontrada")

print("\nSkip-gram similitud -> positive:")
for palabra in palabras_test:
    try:
        print(f"{palabra}: {w2v_skipgram.wv.most_similar(palabra, topn=5)}")
    except:
        print(f"{palabra}: no encontrada")

CBOW similitud -> positive:
psicohistoria: [('psicología', 0.8443653583526611), ('historia', 0.7844677567481995), ('convención', 0.7822109460830688), ('ciencia', 0.7587755918502808), ('predecir', 0.7432090640068054)]
fundacionista: [('soldado', 0.8334208726882935), ('mero', 0.82328200340271), ('modelo', 0.8183872699737549), ('plano', 0.8164640069007874), ('título', 0.8098716735839844)]
espíritu: [('modelo', 0.833525538444519), ('imperio', 0.8241754174232483), ('emperador', 0.8093100786209106), ('acorazado', 0.8026908040046692), ('fundacionista', 0.8019493818283081)]
planeta: [('mundo', 0.7795717716217041), ('reino', 0.7774387001991272), ('sector', 0.7649549245834351), ('sistema', 0.7637091875076294), ('centro', 0.7297593951225281)]

Skip-gram similitud -> positive:
psicohistoria: [('predecir', 0.7623522877693176), ('validez', 0.7321229577064514), ('psicología', 0.7251515984535217), ('convención', 0.7032368183135986), ('leyes', 0.7030066251754761)]
fundacionista: [('converso', 0.7961498

#### Consulta de similitudes negative

Como lo comentamos anteriormente palabras que tiene una distancia vectorial mas alejada de la palabra.

In [110]:
# # Palabras que MENOS se relacionan con ....
# Similitudes con relación negativas, palabras que conceptualmente opuestas o no relacionadas
w2v_skipgram.wv.most_similar(negative=["imperio"], topn=5)
# valores de vectores mas alijados a la palabra. 

[('mientras', 0.02322714403271675),
 ('pausa', -0.02752688154578209),
 ('tenía', -0.0357431136071682),
 ('ojos', -0.04321027174592018),
 ('homir', -0.04343747720122337)]

In [107]:
# # Palabras que MENOS se relacionan con ....
# Similitudes con relación negativas, palabras que conceptualmente opuestas o no relacionadas
w2v_cbow.wv.most_similar(negative=["imperio"], topn=5)
# valores de vectores mas alijados a la palabra. 

[('—dijo', 0.2635144591331482),
 ('—replicó', 0.2433919608592987),
 ('—preguntó', 0.23636683821678162),
 ('—repuso', 0.22727835178375244),
 ('homir', 0.20920304954051971)]

#### Manejo de los vectores de las palabras.

**Obtener los vectores de una palabra**: recordemos que anteriormente se configuro `vector_size=300` en lso 2 modelos por esa razón estamos viendo el vector completo de la palabra "gobernante" con  300 dimensiones. <a href="#Word2Vec">Word2Vec arquitectura CBOW y Skip-gram</a>


In [118]:
#Visualización de vectores en el modelo SKIP-GRAM
#get_vector = w2v_skipgram.wv.get_vector("gobernante")

#w2v_cbow
# Confirma el tamaño:
print("Estructura del vocabulario palabras x dimensiones:", w2v_skipgram.wv.vectors.shape)
# Resultado: (4644, 300)
#            ↑     ↑
#       palabras  dimensiones

print("Cantidad de vectores que tiene 'gobernante':", len(w2v_skipgram.wv['gobernante']))
# Resultado: 300

Estructura del vocabulario palabras x dimensiones: (4644, 300)
Cantidad de vectores que tiene 'gobernante': 300


In [120]:
posicion = w2v_skipgram.wv.key_to_index[palabra]
dimensiones = len(w2v_skipgram.wv[palabra])
total_palabras = len(w2v_skipgram.wv.index_to_key)
    
print(f"La palabra '{palabra}' es la palabra #{posicion + 1} de {total_palabras}, con {dimensiones} dimensiones")
   

La palabra 'gobernante' es la palabra #3064 de 4644, con 300 dimensiones


In [125]:
# modelo SKIP-GRAM
palabra ='imperio'

# el método `get_vector` permite obtener los vectores:
get_vector = w2v_skipgram.wv.get_vector(palabra)
print (f'Palabra {palabra}  posición {w2v_skipgram.wv.key_to_index[palabra]} dimenciones {len(w2v_skipgram.wv[palabra])} Vector :')
print(get_vector)
# Obtener el vector completo mostrando  el vector real de 300 números de "gobernante"
# Estos números no son interpretables individualmente
# Su valor está en las relaciones con otros vectores

# modelo CBOW
#get_vector = w2v_cbow.wv.get_vector("gobernante")
#print(get_vector)

Palabra imperio  posición 56 dimenciones 300 Vector :
[ 0.19409874  0.01573598  0.08015897 -0.03631405  0.15944874 -0.5250009
 -0.3347543   0.4282128  -0.09902585 -0.14503525 -0.06346194  0.02194023
 -0.25882226  0.28304267  0.18101196 -0.32378036 -0.22360295  0.31578523
 -0.18539077 -0.09086666  0.18001708  0.17867883 -0.32029477  0.20244502
 -0.03397013  0.3181885   0.03282682  0.06901656  0.11301483  0.12488827
  0.2008269   0.08412392  0.5394682  -0.06675683  0.09548599  0.26610464
  0.65936494  0.21876372  0.41565597  0.12254391  0.3163139  -0.49817804
 -0.07341616  0.4246808   0.01768071 -0.5370595   0.33464912  0.42576408
  0.26745176  0.5678598  -0.15728584  0.26447973 -0.03237135 -0.12749626
 -0.00546669  0.3469397   0.03653575  0.05815914  0.03152079 -0.1996716
 -0.4110965  -0.15564004  0.07368261  0.22838435 -0.06387999  0.24067177
  0.11521165  0.5796124   0.30342594  0.28354093  0.18727341 -0.2172573
 -0.30145246 -0.18071704  0.18207006  0.45042813 -0.44592774 -0.06757113


In [127]:
print (f'Similitud usando el vector get_vector de la palabra "{palabra}" ')
# = "gobernante"
# el método `most_similar` también permite comparar a partir de vectores
w2v_skipgram.wv.most_similar(get_vector)

Similitud usando el vector get_vector de la palabra "imperio" 


[('imperio', 1.0),
 ('espíritu', 0.6739826202392578),
 ('galáctico', 0.5707152485847473),
 ('universal', 0.5567495822906494),
 ('fundacionista', 0.5410391688346863),
 ('lugar', 0.5362287759780884),
 ('restos', 0.5326494574546814),
 ('modelo', 0.5325555801391602),
 ('nyak', 0.5317217111587524),
 ('interregno', 0.5316949486732483)]

In [128]:
# Aritmética Vectorial en Word2Vec
resultado = w2v_skipgram.wv.get_vector("imperio") - w2v_skipgram.wv.get_vector("galáctico") + w2v_skipgram.wv.get_vector("fundacionista")
w2v_skipgram.wv.most_similar([resultado])

[('imperio', 0.8050969243049622),
 ('fundacionista', 0.6731754541397095),
 ('sumamente', 0.5267733335494995),
 ('converso', 0.5156847238540649),
 ('espíritu', 0.5053003430366516),
 ('—ahora', 0.502896249294281),
 ('—¡qué', 0.499081552028656),
 ('peligroso', 0.4981633126735687),
 ('esto…', 0.49484819173812866),
 ('—…', 0.49283191561698914)]

cuando realizamos operaciones **Aritmética Vectorial en Word2Vec** podemos alterar las relaciones entre las palabras. como podemos ver  en nuestro ejemplo  anterior podemos observar el nivel de relación perteneciente a imperio.

- [('imperio', 1.0),
- ('galáctico', 0.5707152485847473),
-  ('fundacionista', 0.5410391688346863),

al aplicar aritmética vectorial en Word2Vec, donde : 
- [('imperio', 0.8050969243049622),
- ('fundacionista', 0.6731754541397095),
- "imperio" - "galáctico" = alejamos "imperio" y cambiando su valor vectorial de "galáctico"
- "imperio" + "fundacionista" = el nuevo vector de "imperio" lo acerca a "fundacionista" obteniendo un nuevo espacio semántico. 





In [131]:
# función que realiza comparación de palabras 
def comparar_modelos(palabra, skipgram_model, cbow_model, topn=10):
    """
    Compara las similitudes de una palabra en ambos modelos
    """
    print(f"\n=== FUNCIÓN QUE COMPARACIÓN PARA LA PALABRA: '{palabra}' ===")
    
    try:
        # Skip-gram
        sim_skipgram = skipgram_model.wv.most_similar(palabra, topn=topn)
        print(f"\n SKIP-GRAM (sg=1):")
        for i, (word, score) in enumerate(sim_skipgram, 1):
            print(f"  {i:2d}. {word:<15} (similitud: {score:.4f})")
        
        # CBOW
        sim_cbow = cbow_model.wv.most_similar(palabra, topn=topn)
        print(f"\n CBOW (sg=0):")
        for i, (word, score) in enumerate(sim_cbow, 1):
            print(f"  {i:2d}. {word:<15} (similitud: {score:.4f})")
            
        # Palabras en común
        words_sg = set([word for word, _ in sim_skipgram])
        words_cbow = set([word for word, _ in sim_cbow])
        comunes = words_sg.intersection(words_cbow)
        
        print(f"\n PALABRAS EN COMÚN: {len(comunes)}/{topn}")
        print(f"   {list(comunes)}")
        
    except KeyError:
        print(f" La palabra '{palabra}' no está en el vocabulario de uno o ambos modelos")

# Ejemplos de comparación
palabras_test = ["robot", "galaxia", "planeta", "psicohistoria", "robot"]

for palabra in palabras_test:
    comparar_modelos(palabra, w2v_skipgram, w2v_cbow, topn=4)
    print("-" * 80)


=== FUNCIÓN QUE COMPARACIÓN PARA LA PALABRA: 'robot' ===
 La palabra 'robot' no está en el vocabulario de uno o ambos modelos
--------------------------------------------------------------------------------

=== FUNCIÓN QUE COMPARACIÓN PARA LA PALABRA: 'galaxia' ===

 SKIP-GRAM (sg=1):
   1. periferia       (similitud: 0.5833)
   2. gente           (similitud: 0.5492)
   3. galaxia»        (similitud: 0.5451)
   4. opuesto         (similitud: 0.5285)

 CBOW (sg=0):
   1. periferia       (similitud: 0.7901)
   2. humanidad       (similitud: 0.6605)
   3. opuesto         (similitud: 0.6300)
   4. enciclopedia    (similitud: 0.6277)

 PALABRAS EN COMÚN: 2/4
   ['opuesto', 'periferia']
--------------------------------------------------------------------------------

=== FUNCIÓN QUE COMPARACIÓN PARA LA PALABRA: 'planeta' ===

 SKIP-GRAM (sg=1):
   1. reino           (similitud: 0.6766)
   2. esplendor       (similitud: 0.6490)
   3. aislamiento     (similitud: 0.6484)
   4. sector         

### Visualizar agrupación de vectores  (visualización de los embeddings)

In [93]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  
    # reducción de dimensionalidad TSNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [94]:
# !pip install plotly
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_skipgram)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab


In [95]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_skipgram,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [96]:
#import webbrowser
## También se pueden guardar los vectores y labels como tsv para graficar en
## http://projector.tensorflow.org/
#
#modelo = w2v_skipgram
#modelo = w2v_cbow
#
#vectors = np.asarray(modelo.wv.vectors)
#labels = list(modelo.wv.index_to_key)
#
#np.savetxt("vectors.tsv", vectors, delimiter="\t")
#
#with open("labels-metadata.tsv", "w") as fp:
#    for item in labels:
#        fp.write("%s\n" % item)
#
##--------------
### Abrir automáticamente TensorBoard Projector
##webbrowser.open("http://projector.tensorflow.org/")
#
print("1. Ve a: http://projector.tensorflow.org/")

1. Ve a: http://projector.tensorflow.org/
